In [ ]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import connstr
import pandas as pd  

In [ ]:
ldf = pd.read_csv('ldf.csv').loc[:,['col','icn']]
ldf

In [ ]:
engine = create_engine(connstr) 
def get_df(query):
    conn = engine.connect() 
    df = pd.read_sql(query,con=conn) 
    conn.close()
    return df

In [ ]:
def id_split(s):
    s = s + '000'
    s = re.sub('[^\d]', '', s)  
    return s[:4] 
#print( id_split('5.1-4') )

In [ ]:
def id_num(s):
    if s == '':
        return 0
    s = s.strip().lower().replace(' ', '')
    l = ['a','b','c','d','e','f','g','h']
    d = [.100, 1, 100, 10000]
    v = [0,0,0,0]
    for i, c in enumerate(l):
        s = s.replace(c,f'{i}') 
    l = s.split('.')
    for i, c in enumerate(l):
        v[i] =  int(c) / d[i] 
    return sum(v)  
print(id_num('1.1.1')) 
print(id_num('1.1.10.1')) 

In [ ]:
def normalizedf(df):
    df.drop( columns=[c for c in df.columns if '__' in c ] , inplace=True )   
    cols = list(df)   
    df['DATA_TYPE']=df['DATA_TYPE'].str.upper() 
    df['PK_PickListType']=df['PK_PickListType'].fillna(0).astype(int)
    df['CHARACTER_MAXIMUM_LENGTH']=df['CHARACTER_MAXIMUM_LENGTH'].fillna(0).astype(int)
    df[cols] = df[cols].astype(str)
    
    return df

In [ ]:
query = f""" \
SELECT * FROM INFORMATION_SCHEMA.COLUMNS 
LEFT JOIN PickListTypes ON UsageField=COLUMN_NAME
WHERE TABLE_NAME = 'PQCInventory' 
"""  
df  = get_df(query)  
df = normalizedf(df)
df = pd.merge(df,ldf, left_on='COLUMN_NAME', right_on='col')
df[:4]

In [ ]:

txt=[]   
for i,r in df.iterrows():
    plt=''
    PK_PickListType=r['PK_PickListType']
    COLUMN_NAME=r['COLUMN_NAME']
    DATA_TYPE=r['DATA_TYPE']
    CHARACTER_MAXIMUM_LENGTH=r['CHARACTER_MAXIMUM_LENGTH']
    if PK_PickListType != '0':
        plt=f' PK_PickListType="{PK_PickListType}" '
    s=f'<CB:DataField DBColumnName="{COLUMN_NAME}" ImportColumnName="{COLUMN_NAME}" {plt} runat="server"/>'
 
    txt.append(f'{s}')
txt='\n'.join(txt) 
print(txt)

In [ ]:


sql=[]   
for i,r in df.iterrows():
    s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()}({r['CHARACTER_MAXIMUM_LENGTH']}) = NULL"
    if r['DATA_TYPE'] == 'INT':
        s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()} = NULL" 
    sql.append(f',{s}')
sql='\n'.join(sql)[1:]
print(sql)

In [ ]:
dfh = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='1') 
dfh = normalizedf(dfh) 
dfh['QuestionText'] = ''
dfh[0:2] 

In [ ]:
df1 = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='ALL') 
df1 = normalizedf(df1) 
df1[1:2]  # .loc[ df1['IDTEXT'].str.contains('5.') ]

ppl = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='PurposePickList' )
l=[]
c=0
for i,r in ppl.iterrows():
    if i % 2==1: 
        continue
    c=c+1
    IDTEXT = f"1.1.{ c }"
    QuestionText = f"{ r['DisplayValue'] }"
    l.append( {'IDTEXT':f'{IDTEXT}', 'QuestionText': f"If {QuestionText}, describe:"})
ppl = pd.DataFrame(l)  
ppl.to_csv(r'C:\Users\Tim\Documents\1802\HVA\ppl.csv', index=False)  
ppl[0:2]


In [ ]:
dfp = pd.read_csv(r'C:\Users\Tim\Documents\1802\HVA\HVAPurpose.csv' )
dfp['IDTEXT'] = dfp['IDTEXT'].apply(lambda s : s.replace('.a','.1'))
dfp['IDTEXT'] = dfp['IDTEXT'].apply(lambda s : s.replace('.b','.2'))
dfp = normalizedf(dfp)  
dfp[:4]

In [ ]:
dfm = pd.merge(df1, dfp, how='outer', on=['IDTEXT', 'QuestionText'], suffixes=(None, '__')) 
dfm = normalizedf(dfm)
dfm.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfm.csv', index=False)  
dfm[0:2]  

In [ ]:
dfa = dfm
dfa = normalizedf(dfa) 
for i,r in dfa.iterrows():
    v=dfa.loc[i, 'QuestionText']
    if v=='':
        dfa.loc[i, 'QuestionText']=dfa.loc[i, 'QuestionText_']
dfa.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfa.csv', index=False)   
dfa['QuestionText'].fillna('')
dfa[0:2] 

In [ ]:
dff = pd.merge(dfa, dfq, how='left', on='IDTEXT', suffixes=('', '_y')) 
dff['FK_QuestionType'].fillna('9', inplace=True)
dff['FK_PickList'].fillna('NULL', inplace=True)
dff['FK_PickListType'].fillna('NULL', inplace=True)
dff['WarningQuestion'].fillna('NULL', inplace=True)
dff['PK_ExternalLink'].fillna('NULL', inplace=True)
dff['PK_Question']=dff['QPK']
dff['FK_InputType'].fillna('NULL', inplace=True) 
dff['help_text'].fillna('NULL', inplace=True) 
dff = dff.fillna('NULL')
dff  = dff.replace('None','NULL')
dff.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfa.csv', index=False)   
dff[150:]

In [ ]:
query = f"""  SELECT TOP 1 
    PK_Question, FormName, FK_QuestionGroup,sortpos,identifier_text, FK_QuestionType
    , FK_InputType, FK_PickListType, WarningQuestion, ExternalLinkType, QuestionText, help_text 
    FROM fsma_Questions 
    """  
df_qmeta = get_df(query) 
cols = ','.join(df_qmeta.columns)
INS = f'INSERT INTO fsma_Questions ({cols}) VALUES '
print(INS)

In [ ]:
dff = get_df("""
SELECT PK_Question QPK
  , CONVERT(NVARCHAR(9), FK_QuestionType) + '' FK_QuestionType
  , CONVERT(NVARCHAR(9), FK_PickListType) + '' FK_PickListType
  , QTEXT As QuestionText
  , help_text
  , IdText as IDTEXT 
  FROM vwQuestions WHERE QGROUP=2433
  ORDER BY SortPos ASC
""") 
dff=dff.fillna('~')
dff

In [ ]:
sql = "" 
for i, r in dff[:].iterrows(): 
    s = "(@QID, @FormName, @PK_QGroup, @SORD, N'@IDTEXT', @QTYPE, 1, @PKTYPE, 0, NULL, N'@QTEXT','@HTEXT'),"
    s = s.replace('@QID',  str(r['QPK']))
    s = s.replace('@SORD', str(i+1)) 
    s = s.replace('@QTYPE', r['FK_QuestionType'])
    s = s.replace('@PKTYPE', r['FK_PickListType'])
    s = s.replace('@IDTEXT', r['IDTEXT']) 
    s = s.replace('@QTEXT', r['QuestionText'].replace('\'', '`').replace('\ufffd', ''))  
    s = s.replace('@HTEXT', r['help_text'])  
    sql = sql + s + '\n'
sql = sql.replace("'~')", "NULL)")    
# sql = sql.replace("N'NULL'", "N''")   
sql = sql.replace("~,", "NULL,")  
# sql = f'{INS}\n {sql}\n' 
print( sql )

In [ ]:
with open(fr'C:\Users\Tim\Documents\1802\HVA\update.sql', 'w') as f:
    f.write(sql)  



df = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='PurposePickList').fillna('NULL') 
cols = list(df) # Creates list of all column headers
df[cols] = df[cols].astype(str)
df['DisplayCode'] =  df['DisplayValue'].apply(lambda s: re.sub('[^A-Z0-9]','',s.upper()))
df['DisplayCode'] =  df['DisplayCode'].apply(lambda s: re.sub('[AEIOURST]','',s)[:8])
s =''
for i,r, in df.iterrows():
    PK_PickList=r['PK_PickList']
    DisplayCode=r['DisplayCode']
    DisplayValue=r['DisplayValue'] 
    if DisplayValue=='Other':
        DisplayCode='OTH'
    s = s + f"({PK_PickList},@PKT, N'{DisplayCode}',N'{DisplayValue}',{i}, GETDATE(), 1),\n"

print(s)


In [ ]:
query = f""" \
    SELECT TOP 5000 
      IDTEXT AS IDTEXT 
    , PK_Question 
    FROM vwQuestions WHERE PK_FORM LIKE '2022-A-HVA'  
"""  
dfdep = get_df(query)  
# dfdep.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfdep.csv', index=False)  
dfdep[:] 

In [ ]:
INS = f"""  INSERT INTO [fsma_QuestionDependencies] 
(FK_Question, FK_Question_Master, Question_Master_CodeValue_ToMakeRequired) VALUES 
    """  
sql = ""   
for i, r in dfdep[:].iterrows(): 
    mast = 28211
    s = f"({r['PK_Question']}, {mast}, 'Y') ,"     
    sql = sql + s + '\n' 
sql = f'{INS}\n {sql}\n' 
# print(sql)